In [ ]:
# Imports
import pandas as pd
import numpy as np
import sys
sys.path.append('..')

from load_data import load_dataset
import google.generativeai as genai
from insights.prompt_templates import *
import json
from datetime import datetime
import os
from utils.preprocessing import prepare_reddit_for_llm, create_statistical_summary_for_llm
from utils.evaluation import analyze_comprehensive_mental_health
from dotenv import load_dotenv
load_dotenv()

In [ ]:
# Configure Gemini
genai.configure(api_key=os.environ.get("GEMINI_API_KEY"))
model = genai.GenerativeModel('gemini-pro')

print("Gemini model loaded successfully")

In [ ]:
# Load Reddit Data
reddit_df = load_dataset('reddit')

# Load CDC data
cdc_df = load_dataset('cdc')

# Load WHO Suicide data
who_suicide_df = load_dataset('who_suicide')

# Load Mental Health Care data
mental_health_care_df = load_dataset('mental_health_care')

In [ ]:
# Prepare Reddit sample
reddit_sample = prepare_reddit_for_llm(reddit_df)

# Generate statistical summary
statistical_data = create_statistical_summary_for_llm(cdc_df, who_suicide_df, mental_health_care_df)

# Format statistical data for LLM
stats_text = ""
for dataset, stats in statistical_data.items():
    stats_text += f"\n{dataset}:\n"
    for key, value in stats.items():
        stats_text += f"- {key}: {value}\n"


In [ ]:
# Run analysis with LLM
comprehensive_analysis = analyze_comprehensive_mental_health(reddit_sample, stats_text, model)

if comprehensive_analysis:
    print("\n" + "="*80)
    print("COMPREHENSIVE MENTAL HEALTH DATA ANALYSIS")
    print("="*80)
    print(comprehensive_analysis)
else:
    print("Analysis failed")

In [ ]:
# Save analysis results
if comprehensive_analysis:
    # Save analysis to outputs
    os.makedirs('../outputs/reports', exist_ok=True)
    
    # Create comprehensive report with metadata from all sources
    report = {
        'timestamp': datetime.now().isoformat(),
        'analysis_type': 'comprehensive_mental_health',
        'data_sources': {
            'reddit_sample_size': len(reddit_sample.split('---'))-1,
            'statistical_sources': list(statistical_data.keys()),
            'cdc_records': len(cdc_df) if len(cdc_df) > 0 else 0,
            'who_records': len(who_suicide_df) if len(who_suicide_df) > 0 else 0,
            'care_records': len(mental_health_care_df) if len(mental_health_care_df) > 0 else 0
        },
        'analysis': comprehensive_analysis,
        'data_summary': statistical_data
    }
    
    filename = f"../outputs/reports/comprehensive_mental_health_{datetime.now().strftime('%Y%m%d')}.json"
    
    with open(filename, 'w') as f:
        json.dump(report, f, indent=2, default=str)

else:
    print("No analysis to save - comprehensive analysis failed")